<div style="display: flex; align-items: center;padding-bottom:20px;">
    
    <div>    
        <h1 style="margin: 0;">Data Scientist Consultant – Commodity Insights</h1>
        <h3 style="margin-top: 5px;">Technical Assessment</h3>
    </div>
</div>
<div >        
    <h4 style="margin-top: 5px;">Presentend by Jairo Ruiz Saenz - April 21st, 2024</h4>
    <p style="font-size: 15px;">
        This notebook provides a comprehensive analysis of the data challenge, detailing all steps from data preprocessing and cleaning to extracting insights and addressing business questions.
    </p>
</div>

## Technical Assessment

- **Objective**: Build a model to estimate production
- **Dataset**: http://huy302.github.io/interview_dataset.csv
- **Features description**: https://huy302.github.io/feature_desc.md
- **Problem description**: Production prediction is one of the core problems in our business. The provided dataset is a set of nearby wells located in the United States and their 12 months cumulative production. As a data scientist you want to build a model from scratch to predict production and show your manager that your model can perform well on unseen data.
- **Submitting material**: code (Python/notebook) and supporting materials if needed (analysis, documentation, paper, slide)

## Dataset feature descriptions

- **treatment company**: The treatment company who provides treatment service.
- **azimuth**: Well drilling direction.
- **md (ft)**: Measure depth.
- **tvd (ft)**: True vertical depth.
- **date on production**: First production date.
- **operator**: The well operator who performs drilling service.
- **footage lateral length**: Horizontal well section.
- **well spacing**: Distance to the closest nearby well.
- **porpoise deviation**: How much max (in ft.) a well deviated from its horizontal.
- **porpoise count**: How many times the deviations (porpoises) occurred.
- **shale footage**: How much shale (in ft) encountered in a horizontal well.
- **acoustic impedance**: The impedance of a reservoir rock (ft/s * g/cc).
- **log permeability**: The property of rocks that is an indication of the ability for fluids (gas or liquid) to flow through rocks
- **porosity**: The percentage of void space in a rock. It is defined as the ratio of the volume of the voids or pore space divided by the total volume. It is written as either a decimal fraction between 0 and 1 or as a percentage.
- **poisson ratio**: Measures the ratio of lateral strain to axial strain at linearly elastic region.
- **water saturation**: The ratio of water volume to pore volume.
- **toc**: Total Organic Carbon, indicates the organic richness (hydrocarbon generative potential) of a reservoir rock.
- **vcl**: The amount of clay minerals in a reservoir rock.
- **p-velocity**: The velocity of P-waves (compressional waves) through a reservoir rock (ft/s).
- **s-velocity**: The velocity of S-waves (shear waves) through a reservoir rock (ft/s).
- **youngs modulus**: The ratio of the applied stress to the fractional extension (or shortening) of the reservoir rock parallel to the tension (or compression) (giga pascals).
- **isip**: When the pumps are quickly stopped, and the fluids stop moving, these friction pressures disappear and the resulting pressure is called the instantaneous shut-in pressure, ISIP.
- **breakdown pressure**: The pressure at which a hydraulic fracture is created/initiated/induced.
- **pump rate**: The volume of liquid that travels through the pump in a given time. A hydraulic fracture is formed by pumping fluid into a wellbore at a rate sufficient to increase pressure at the target depth, to exceed that of the fracture gradient (pressure gradient) of the rock.
- **total number of stages**: Total stages used to fracture the horizontal section of the well.
- **proppant volume**: The amount of proppant in pounds used in the completion of a well (lbs).
- **proppant fluid ratio**: The ratio of proppant volume/fluid volume (lbs/gallon).
- **production**: The 12 months cumulative gas production (mmcf - million cubic feet).

In [1]:
# Import of libraries used in the script

import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

In [2]:
# Set display options to ensure all columns and rows are displayed when using functions like df.head()

pd.set_option('display.max_columns', None)  # Set maximum number of columns to display to None (unlimited)
pd.set_option('display.max_rows', 100)  # Set maximum number of rows to display to None (unlimited)
pd.set_option('display.precision', 2)  # Set precision for float numbers to 2 decimal places
pd.set_option('display.max_colwidth', None)  # Set maximum column width to None (unlimited)

In [3]:
# Reads the CSV file using it's url and create a DataFrame

data_url = 'http://huy302.github.io/interview_dataset.csv'
df = pd.read_csv(data_url)

In [4]:
df.shape

(1000, 28)

In [5]:
# Preview of the DataFrame

df.sample(n=5)

,treatment company,azimuth,md (ft),tvd (ft),date on production,operator,footage lateral length,well spacing,porpoise deviation,porpoise count,shale footage,acoustic impedance,log permeability,porosity,poisson ratio,water saturation,toc,vcl,p-velocity,s-velocity,youngs modulus,isip,breakdown pressure,pump rate,total number of stages,proppant volume,proppant fluid ratio,production
726,treatment company 12,-2.07,13325,8105.0,1/1/2013,operator 7,4467.0,4766.46,0.22,10,5793,34734.62,0.25,0.02,0.34,0.86,4.76,0.53,13275.97,6918.17,30.79,5560.0,NaN,88,31,1.98e+07,3.02,2103.99
834,treatment company 5,-17.05,12610,6966.0,1/1/2014,operator 24,5066.0,1018.77,27.21,7,5088,30838.42,0.26,0.02,0.33,NaN,4.68,0.61,13239.24,6958.72,30.94,4353.0,NaN,80,15,5.21e+06,1.33,1312.63
846,treatment company 11,-36.27,11886,7118.0,5/1/2015,operator 14,4127.0,2124.69,4.11,5,1357,30837.96,1.00,0.02,0.28,0.69,4.68,0.47,11558.58,6768.67,28.71,NaN,7710.0,80,21,7.99e+06,1.06,2991.75
119,treatment company 12,-40.20,15240,8482.0,1/1/2013,operator 9,6297.0,758.84,5.00,3,2288,35105.97,0.96,0.06,0.29,0.25,4.30,0.63,11831.43,7085.47,31.91,6537.0,NaN,90,22,8.62e+06,1.17,1127.19
434,treatment company 2,-32.43,16361,8115.0,7/1/2012,operator 20,7551.0,2038.41,2.85,7,4433,34179.95,0.31,0.02,0.33,0.80,4.66,0.61,13182.77,7015.94,31.11,4688.0,NaN,71,14,5.04e+06,1.16,1430.78


In [6]:
# Checking the data types of the DataFrame

df.dtypes

treatment company          object
azimuth                   float64
md (ft)                     int64
tvd (ft)                  float64
date on production         object
operator                   object
footage lateral length    float64
well spacing              float64
porpoise deviation        float64
porpoise count              int64
shale footage               int64
acoustic impedance        float64
log permeability          float64
porosity                  float64
poisson ratio             float64
water saturation          float64
toc                       float64
vcl                       float64
p-velocity                float64
s-velocity                float64
youngs modulus            float64
isip                      float64
breakdown pressure        float64
pump rate                   int64
total number of stages      int64
proppant volume           float64
proppant fluid ratio      float64
production                float64
dtype: object

In [7]:
# Casting the type of the variable "date on production" to datetime

df["date on production"] = pd.to_datetime(df["date on production"], format="%m/%d/%Y", errors="coerce")

In [8]:
# Extracting the year and month form the variable "date on production" and create two new columns

df["date_on_production_year"] = df["date on production"].dt.year
df["date_on_production_month"] = df["date on production"].dt.month

In [9]:
# Casting the type of variables "treatment company" and "operator" to category

df["treatment company"] = df["treatment company"].astype("category")
df["operator"] = df["operator"].astype("category")

In [10]:
# In orther to better understand the data distribution and patterns let's create a profiling report of the DataFrame

profile = ProfileReport(df, title=f"Profiling Report", minimal=True)
profile.to_file(f"profile_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
profile

In [12]:
filtered_df = df.copy()

In [13]:
# Considering the data distribution, we aim to remove outliers in the 'proppant volume' variable.

percentile_lower = df['proppant volume'].quantile(0.15)
percentile_upper = df['proppant volume'].quantile(0.95)

print(f'percentile_lower: {percentile_lower}')
print(f'percentile_upper: {percentile_upper}')

filter = (filtered_df['proppant volume'] >= percentile_lower) & (filtered_df['proppant volume'] <= percentile_upper)
filtered_df = filtered_df[filter]

percentile_lower: 4938275.849999999
percentile_upper: 22971083.2


In [14]:
# Considering the data distribution, we aim to remove outliers in the ''md (ft)' variable.

percentile_lower = filtered_df['md (ft)'].quantile(0.01)
percentile_upper = filtered_df['md (ft)'].quantile(0.99)

print(f'percentile_lower: {percentile_lower}')
print(f'percentile_upper: {percentile_upper}')

filter = (filtered_df['md (ft)'] >= percentile_lower) & (filtered_df['md (ft)'] <= percentile_upper)
filtered_df = filtered_df[filter]

percentile_lower: 10041.28
percentile_upper: 19952.2


In [15]:
# We define the training variables X for the model, I'm leaving out the variables 'date on production'
# because the model cannot handle datetime variables and 'breakdown pressure' has 74.4% of missing values

data_model_x = filtered_df[['treatment company', 'azimuth', 'md (ft)', 'tvd (ft)', 'operator',
 'footage lateral length', 'well spacing', 'porpoise deviation', 'porpoise count', 'shale footage', 'acoustic impedance',
 'log permeability', 'porosity', 'poisson ratio', 'water saturation', 'toc', 'vcl', 'p-velocity', 's-velocity', 'youngs modulus',
 'isip', 'pump rate', 'total number of stages', 'proppant volume', 'proppant fluid ratio', 'date_on_production_year',
 'date_on_production_month']]

# I define the target variable Y

data_model_y = filtered_df['production']

In [16]:
# I transform the 'treatment company' and 'operator' to dummie variables so they can be used in the model

data_model_X = pd.get_dummies(data_model_x, columns=['treatment company', 'operator'])

X = data_model_X
y = data_model_y

# Spliting the DataFrame into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# I define the model to be used, is this case is a Random Forest model and I'm using a random_state in order to be able to replica the results
RF_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Training the model
RF_model.fit(X_train, y_train)

# Testing the model
train_score = RF_model.score(X_train, y_train)
test_score = RF_model.score(X_test, y_test)

# Predicting on test set
y_pred = RF_model.predict(X_test)

# Calculating MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculating RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Cross-validation
cv_scores = cross_val_score(RF_model, X, y, cv=10)

In [17]:
# Printing metrics results

print(f"Training R^2 Score: {train_score:.4f}")
print(f"Test R^2 Score: {test_score:.4f}")
print(f"MAPE: {mape:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"Mean Cross-Validation Score: {np.mean(cv_scores):.4f}")

Training R^2 Score: 0.9403
Test R^2 Score: 0.6825
MAPE: 0.3662
RMSE: 671.6790
Mean Cross-Validation Score: 0.5967


In [18]:
# One of the reasons for choosing the random forest model is its ability
# to identify important variables that affect the prediction of the property value.
# Below are the 10 most important features

feature_importances = RF_model.feature_importances_

# Sorts the variable importances from highest to lowest
sorted_indices = np.argsort(feature_importances)[::-1]

# Names of the variables corresponding to the sorted importances
feature_names = X_train.columns[sorted_indices]

# Sorted variable importances
sorted_feature_importances = feature_importances[sorted_indices]

feature_importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': sorted_feature_importances})
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

pd.set_option('display.float_format', lambda x: '%.5f' % x)
feature_importances_df.head(10)

,Feature,Importance
0,proppant volume,0.19674
1,tvd (ft),0.12385
2,youngs modulus,0.08620
3,operator_operator 14,0.08105
4,md (ft),0.04643
5,operator_operator 6,0.04439
6,total number of stages,0.03449
7,p-velocity,0.03352
8,isip,0.03320
9,well spacing,0.02540
